In [2]:
from dotenv import load_dotenv

load_dotenv()

True

### 데이터를 효과적으로 전달하는 방법

    - `RunnablePassthrough` : 입력을 변경하지 않거나 추가 키를 더하여 그대로 전달할 수 있음
    - `RunnablePassthrough()` 가 단독으로 호출되면, 단순히 입력을 받아 그대로 전달함
    - `RunnablePassthrough.assign(...)` 방식으로 호출되면, 입력을 받아 assign 함수에 전달된 추가 인수를 추가함

In [3]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

In [4]:
prompt = PromptTemplate.from_template("{num}의 10배는?")
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

chain = prompt | llm

In [5]:
chain.invoke({"num": 5})

AIMessage(content='50', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 15, 'total_tokens': 16, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-8ed8eb68-5324-43d5-a0bd-63056bdfaf29-0', usage_metadata={'input_tokens': 15, 'output_tokens': 1, 'total_tokens': 16})

In [6]:
chain.invoke(5)

AIMessage(content='50', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 15, 'total_tokens': 16, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-58187092-921e-4c5b-914f-bc653a44f38f-0', usage_metadata={'input_tokens': 15, 'output_tokens': 1, 'total_tokens': 16})

In [10]:
# RunnablePassthrough를 사용한 예제 (runnable 객체이며 runnable 객체는 invoke() 메소드를 사용해 별도 실행 가능)


from langchain_core.runnables import RunnablePassthrough

RunnablePassthrough().invoke({"num":10})

{'num': 10}

In [11]:
result = RunnablePassthrough().invoke({"num":10})
type(result)

dict

In [12]:
runnable_chain = {"num" : RunnablePassthrough()} | prompt | ChatOpenAI(model='gpt-3.5-turbo')

# dict 값이 RunnablePassthrough()로 변경됨

runnable_chain.invoke(10)

AIMessage(content='100', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 15, 'total_tokens': 16, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-30541df7-c363-413b-8bce-00d6ec292916-0', usage_metadata={'input_tokens': 15, 'output_tokens': 1, 'total_tokens': 16})

In [13]:
RunnablePassthrough().invoke({"num":1})

{'num': 1}

#### `RunnablePassthrough.assgin()`
    - 입력 값으로 들어온 값의 key/value 값을 새롭게 할당된 key/value 값을 합침 

In [14]:
# RunnablePassthrough.assign()을 사용하는 경우와 비교
# 입력 키 : num, 할당(assign) 키 : new_num

(RunnablePassthrough.assign(new_num=lambda x: x["num"] * 3)).invoke({"num" : 1})

{'num': 1, 'new_num': 3}

#### RunnableParallel

    병렬로 Runnable을 실행하는 RunnableParallel

In [16]:
from langchain_core.runnables import RunnableParallel

# RunnableParallel 인스턴스 생성. 이 인스턴스는 여러 Runnable 인스턴스를 병렬로 실행

runnable = RunnableParallel(
    passed=RunnablePassthrough(), # RunnablePassthrough 인스턴스를 'passed' 키워드 인자로 전달. 입력된 데이터를 그대로 통과시킴
    extra = RunnablePassthrough.assign(mult=lambda x : x["num"]*3), # 'extra' 키워드 인자로 RunnablePassthrough.assign을 사용해 'multi' 람다 함수 할당. 이 함수는 입력된 딕셔너리의 'num' 키에 해당하는 값을 3배로 증가
    modified = lambda x: x["num"] + 1, # 'modified' 키워드 인자로 람다 함수 전달. 이 함수는 입력된 딕셔너리의 'num' 키에 해당하는 값에 1을 더함
    
)

runnable.invoke({"num":1})

{'passed': {'num': 1}, 'extra': {'num': 1, 'mult': 3}, 'modified': 2}

In [17]:
# chain도 RunnableParallel 적용할 수 있음

chain1 = (
    {"country" : RunnablePassthrough()}
    | PromptTemplate.from_template("{country}의 수도는?")
    | ChatOpenAI(model_name='gpt-3.5-turbo')
)

chain2 = (
    {"country" : RunnablePassthrough()}
    | PromptTemplate.from_template("{country}의 면적은?")
    | ChatOpenAI(model_name="gpt-3.5-turbo")
)

In [18]:
combined_chain = RunnableParallel(captical=chain1, area=chain2)
combined_chain.invoke("대한민국")

{'captical': AIMessage(content='서울특별시입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 18, 'total_tokens': 28, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-8f878394-25b8-4f19-98fc-b9880c3bf7ee-0', usage_metadata={'input_tokens': 18, 'output_tokens': 10, 'total_tokens': 28}),
 'area': AIMessage(content='대한민국의 총 면적은 약 100,363제곱킬로미터 입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 19, 'total_tokens': 50, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-dbba75f7-7927-4cb7-8357-e2675d7b3969-0', usage_metadata={'input_tokens': 19, 'output_tokens': 31, 'total_tokens': 50})}

#### RunnableLambda

    - 사용자 정의 함수를 매핑

In [19]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from datetime import datetime

In [22]:
def get_today():
    return datetime.today().strftime("%b-%d")


get_today()

'Sep-27'

In [23]:
def get_today(a):
    return datetime.today().strftime("%b-%d")


get_today(None)

'Sep-27'

In [24]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

prompt = PromptTemplate.from_template(
    "{today}가 생일인 유명인 {n}명을 나열하세요. 생년월일을 표기해 주세요."
)

llm = ChatOpenAI(model_name='gpt-3.5-turbo')

chain = (
    {"today" : RunnableLambda(get_today), "n" : RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()

)

In [25]:
print(chain.invoke(3))

1. 윤아 (Girls' Generation 멤버) - 1990년 9월 27일
2. 루이스 해밀턴 (F1 레이서) - 1985년 9월 27일
3. 게이리 콜 (영국 배우) - 1967년 9월 27일
